In [1]:
'heloo'

'heloo'

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BertTokenizer, BertForMaskedLM

def gpt2_generate_response(input_text):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids, max_length=100, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

def bert_generate_text(prompt):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertForMaskedLM.from_pretrained("bert-base-uncased")
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

d:\Work\Job\RefonteInfiniti.com\GitHub Projects\Demos\Week4Sesson1\backend_data_science\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
user_input = 'Hello, how are you?'
response = gpt2_generate_response(user_input)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, how are you?

I'm a little bit of a nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd


In [7]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

def predict_masked_word(sentence):
    # Load pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertForMaskedLM.from_pretrained("bert-base-uncased")
    
    # Tokenize input sentence and find the index of the masked token
    input_ids = tokenizer.encode(sentence, return_tensors="pt")
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
    
    # Predict the masked token
    with torch.no_grad():
        output = model(input_ids)
    
    # Get the logits for the masked token and find the top prediction
    mask_token_logits = output.logits[0, mask_token_index, :]
    top_token_id = torch.argmax(mask_token_logits, dim=-1)
    
    # Decode the predicted token ID back to a word
    predicted_token = tokenizer.decode(top_token_id)
    return predicted_token

# Example usage
sentence = "Hello [MASK]?"
predicted_word = predict_masked_word(sentence)
print(f"Predicted word: {predicted_word}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Predicted word: there


In [14]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def gpt2_generate_response(input_text):
    # Load pre-trained GPT-2 model and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    
    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    
    # Generate text
    output = model.generate(input_ids, max_length=65, num_return_sequences=1)
    
    # Decode generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return generated_text

# Example usage
input_text = "Once upon a time, in a land far away, there was a small village where everyone lived in harmony."
generated_text = gpt2_generate_response(input_text)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, in a land far away, there was a small village where everyone lived in harmony. The village was called the "Bamboo Village" and it was a place where people could live together.

The village was called the "Bamboo Village" because it was the place where the people lived together


In [15]:
!pip install langchain

     -------------------------------------- 973.6/973.6 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 3.7 MB/s eta 0:00:00
  Using cached aiohttp-3.9.5-cp311-cp311-win_amd64.whl (370 kB)
     -------------------------------------- 310.4/310.4 kB 3.8 MB/s eta 0:00:00
     -------------------------------------- 124.6/124.6 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 409.6/409.6 kB 3.6 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
  Using cached frozenlist-1.4.1-cp311-cp311-win_amd64.whl (50 kB)
  Using cached multidict-6.0.5-cp311-cp311-win_amd64.whl (28 kB)
  Using cached yarl-1.9.4-cp311-cp311-win_amd64.whl (76 kB)
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
     -------------------------------------- 138.8/138.8 kB 8.6 MB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 3.3 MB/s eta 0:00:00
     -------------------


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
# from langchain.chains import SingleShot
# from langchain.schema import LanguageModelSchema
# from langchain.llms import HuggingFaceModel

# def run_lang_chain():
#     # Initialize the Hugging Face model
#     model = HuggingFaceModel(model_name='distilgpt2')

#     # Set up a single shot chain with the model
#     single_shot = SingleShot(model)

#     # Use the chain to answer a query
#     query = "Explain how blockchain technology works."
#     answer = single_shot.run(query)
#     print("Answer:", answer)

# if __name__ == "__main__":
#     run_lang_chain()

In [19]:
import os
# from langchain.vectorstores import FAISS
# from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.chains import RetrievalQA
# from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.chains import RetrievalQAWithSourcesChain
from huggingface_hub import notebook_login
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
# from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
import textwrap
import sys
import os